In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

train = pd.read_csv('../input/playground-series-s4e5/train.csv',index_col="id")
test = pd.read_csv('../input/playground-series-s4e5/test.csv',index_col="id")

In [3]:
def cleaning(dataset):
    features = dataset.columns.tolist()

    dataset['mean_features'] = 0.1*dataset[features].mean(axis=1)
    dataset['std_features'] = dataset[features].std(axis=1)
    dataset['max_features'] = dataset[features].max(axis=1)
    dataset['min_features'] = dataset[features].min(axis=1)
    dataset['median_features'] = 0.1*dataset[features].median(axis=1)
    # just keep the descriptive statistics
    dataset = dataset.drop(features, axis=1)

    return dataset

In [4]:
target = "FloodProbability"
randomState = 41
n_splits = 5

X = train.drop(target, axis=1)
X = cleaning(X)
y = train[target]

scaler = StandardScaler()
scaler.fit(X)

X = pd.DataFrame(scaler.transform(X), columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=randomState)
kf = KFold(n_splits=n_splits, shuffle=True, random_state=randomState)

# testScaled = pd.DataFrame(scaler.transform(testDropped), columns=testDropped.columns)

In [5]:
cleanedTest = test
cleanedTest = cleaning(cleanedTest)
cleanedTest = pd.DataFrame(scaler.transform(cleanedTest), columns=cleanedTest.columns)
cleanedTest

,mean_features,std_features,max_features,min_features,median_features
0,1.470008,-1.211884,-0.967722,1.671050,1.280758
1,-1.057894,-0.398030,-0.201344,-0.723063,-1.376934
2,-1.057894,0.559958,-0.201344,-0.723063,-0.491036
3,-0.817142,-1.218548,-0.967722,0.473994,-1.376934
4,-0.817142,-1.218548,-0.967722,0.473994,-0.491036
...,...,...,...,...,...
745300,-0.576389,-1.619421,-0.967722,-0.723063,0.394861
745301,-1.178270,0.099567,2.097788,0.473994,-1.376934
745302,2.553395,0.776859,1.331411,1.671050,0.394861
745303,0.988503,-0.266960,-0.967722,0.473994,2.166655


## LGBM

In [6]:
from lightgbm import LGBMRegressor

lgbm_params = {
        'num_leaves': 100, 
        'learning_rate': 0.015257139375608821, 
        'n_estimators': 1000, 
        'subsample_for_bin': 298999, 
        'min_child_samples': 77, 
        'reg_alpha': 2.7240879929233532e-05, 
        'reg_lambda': 9.693029775370852e-10, 
        'colsample_bytree': 0.8107657422421071, 
        'subsample': 0.8727733774586144, 
        'max_depth': 10,
        'objective':'regression',
        'random_state': randomState, 
        'verbose': -1,  # これを指定しないと`No further splits with positive gain, best gain: -inf`というWarningが表示される
        'early_stopping_round': 10,  # ここでearly_stoppingを指定
        'metric': 'rmse',
    }
    
lgbm_model = LGBMRegressor(**lgbm_params)

lgbm_model.fit(X_train, y_train,
               eval_set=[(X_test, y_test)])

LGBMRegressor(colsample_bytree=0.8107657422421071, early_stopping_round=10,
              learning_rate=0.015257139375608821, max_depth=10, metric='rmse',
              min_child_samples=77, n_estimators=1000, num_leaves=100,
              objective='regression', random_state=41,
              reg_alpha=2.7240879929233532e-05,
              reg_lambda=9.693029775370852e-10, subsample=0.8727733774586144,
              subsample_for_bin=298999, verbose=-1)

In [7]:
predLgbmTest = lgbm_model.predict(X_test)
scoreLGBM = r2_score(y_test, predLgbmTest)
print(f'r2_score: {scoreLGBM: .4f}')

# predLgbm = lgbm_model.predict(testScaled)
predLgbm = lgbm_model.predict(cleanedTest)
print(predLgbm)

r2_score:  0.8693
[0.57942724 0.45496697 0.44940562 ... 0.62095276 0.54913464 0.52942847]


## xgboost

In [8]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)

params = {
        'objective': 'reg:squarederror', 
        'random_state': randomState, 
        'max_depth': 4,
        'learning_rate': 0.04
}

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
modelXGB = xgb.train(
            params, 
            dtrain, 
            #学習のラウンド数
            num_boost_round=1000,
            early_stopping_rounds=10, 
            evals=watchlist
        )

[0]	train-rmse:0.04927	eval-rmse:0.04934
[1]	train-rmse:0.04760	eval-rmse:0.04767
[2]	train-rmse:0.04601	eval-rmse:0.04608
[3]	train-rmse:0.04450	eval-rmse:0.04457
[4]	train-rmse:0.04306	eval-rmse:0.04312
[5]	train-rmse:0.04168	eval-rmse:0.04175
[6]	train-rmse:0.04036	eval-rmse:0.04043
[7]	train-rmse:0.03911	eval-rmse:0.03918
[8]	train-rmse:0.03792	eval-rmse:0.03799
[9]	train-rmse:0.03680	eval-rmse:0.03686
[10]	train-rmse:0.03572	eval-rmse:0.03578
[11]	train-rmse:0.03471	eval-rmse:0.03476
[12]	train-rmse:0.03374	eval-rmse:0.03380
[13]	train-rmse:0.03282	eval-rmse:0.03288
[14]	train-rmse:0.03195	eval-rmse:0.03200
[15]	train-rmse:0.03113	eval-rmse:0.03118
[16]	train-rmse:0.03035	eval-rmse:0.03040
[17]	train-rmse:0.02962	eval-rmse:0.02966
[18]	train-rmse:0.02892	eval-rmse:0.02896
[19]	train-rmse:0.02826	eval-rmse:0.02831
[20]	train-rmse:0.02764	eval-rmse:0.02769
[21]	train-rmse:0.02706	eval-rmse:0.02710
[22]	train-rmse:0.02651	eval-rmse:0.02655
[23]	train-rmse:0.02599	eval-rmse:0.02603
[2

In [9]:
predXGBTest = modelXGB.predict(dvalid)
scoreXGB = r2_score(y_test, predXGBTest)
print(f'r2_score: {scoreXGB: .4f}')

# predXGB = modelXGB.predict(xgb.DMatrix(testScaled))
predXGB = modelXGB.predict(xgb.DMatrix(cleanedTest))
print(predXGB)

r2_score:  0.8693
[0.5787353  0.45433703 0.4500091  ... 0.6201285  0.5493838  0.52942026]


## catBoost

In [10]:
from catboost import Pool
from catboost import CatBoostRegressor

catboost_params ={'depth': 3,                   
                  'random_state':randomState,
                  'task_type': 'CPU', 
                  'eval_metric': 'RMSE', 
                  'min_data_in_leaf': 4, 
                  'loss_function': 'RMSE',
                  'grow_policy': 'Lossguide', 
                  'bootstrap_type': 'Bernoulli',
                  'subsample': 0.83862137638162, 
                  'l2_leaf_reg': 8.365422739510098, 
                  'random_strength': 3.296124856352495, 
                  'learning_rate': 0.0983, 
}
    
modelCatBoost = CatBoostRegressor(**catboost_params)
modelCatBoost.fit(
            X_train,
            y_train,
            eval_set=(X_test, y_test),    # 検証用データ
            early_stopping_rounds=10,  # 10回以上精度が改善しなければ中止
        )

0:	learn: 0.0468598	test: 0.0469299	best: 0.0469299 (0)	total: 196ms	remaining: 3m 15s
1:	learn: 0.0431615	test: 0.0432295	best: 0.0432295 (1)	total: 320ms	remaining: 2m 39s
2:	learn: 0.0399099	test: 0.0399743	best: 0.0399743 (2)	total: 438ms	remaining: 2m 25s
3:	learn: 0.0370035	test: 0.0370673	best: 0.0370673 (3)	total: 555ms	remaining: 2m 18s
4:	learn: 0.0344782	test: 0.0345388	best: 0.0345388 (4)	total: 674ms	remaining: 2m 14s
5:	learn: 0.0322608	test: 0.0323186	best: 0.0323186 (5)	total: 791ms	remaining: 2m 10s
6:	learn: 0.0302711	test: 0.0303293	best: 0.0303293 (6)	total: 914ms	remaining: 2m 9s
7:	learn: 0.0285713	test: 0.0286274	best: 0.0286274 (7)	total: 1.04s	remaining: 2m 8s
8:	learn: 0.0271236	test: 0.0271764	best: 0.0271764 (8)	total: 1.15s	remaining: 2m 6s
9:	learn: 0.0258619	test: 0.0259091	best: 0.0259091 (9)	total: 1.26s	remaining: 2m 4s
10:	learn: 0.0247455	test: 0.0247919	best: 0.0247919 (10)	total: 1.37s	remaining: 2m 3s
11:	learn: 0.0237982	test: 0.0238429	best: 0.0

In [11]:
predCatBoostTest = modelCatBoost.predict(X_test)
scoreCatBoost = r2_score(y_test, predCatBoostTest)
print(f'r2_score: {scoreCatBoost: .4f}')

# predCatBoost = modelCatBoost.predict(testScaled)
predCatBoost = modelCatBoost.predict(cleanedTest)
print(predCatBoost)

r2_score:  0.8692
[0.57820837 0.45362714 0.44948342 ... 0.61993853 0.54911495 0.52910743]


## mix

In [12]:
mixPredTest = (predCatBoostTest * 0.2) + (predLgbmTest * 0.5) + (predXGBTest * 0.3)
print(mixPredTest)
scoreMix = r2_score(y_test, mixPredTest)
print(f'r2_score: {scoreMix: .4f}')

[0.47465099 0.46784149 0.53663078 ... 0.47630835 0.4747097  0.48549624]
r2_score:  0.8694


In [13]:
mixPred = (predCatBoost * 0.2) + (predLgbm * 0.5) + (predXGB * 0.3)

## submit

In [14]:
submit = pd.read_csv('../input/playground-series-s4e5/test.csv')[['id']]
submit['FloodProbability'] = mixPred
print(submit)

             id  FloodProbability
0       1117957          0.578976
1       1117958          0.454510
2       1117959          0.449602
3       1117960          0.466373
4       1117961          0.466763
...         ...               ...
745300  1863257          0.474901
745301  1863258          0.444780
745302  1863259          0.620503
745303  1863260          0.549205
745304  1863261          0.529362

[745305 rows x 2 columns]


In [15]:
fileName = 'submission4.csv'
submit.to_csv(fileName, index=False)
print("Your submission was successfully saved!  : " + fileName)

Your submission was successfully saved!  : submission4.csv
